<a href="https://colab.research.google.com/github/YEONOC/2022-1-AlphaProject/blob/main/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla V100-SXM2-16GB


In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [ ]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
PATH ="/content/drive/MyDrive/Model3(Text Generation)/model3_61.pt"

In [ ]:
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
import pandas as pd
# 감성대화말뭉치(원시데이터) training(만갠줄 알았는데 7만개 .. 정렬 잘못됨..) 
model3_data = pd.read_csv('/content/drive/MyDrive/asset/train_data/감성대화말뭉치(원시데이터)_Training.csv')
model3_data

,번호,value,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3
0,9999,1,청년,여성,"연애, 결혼, 출산",해당없음,분노,악의적인,제가 사랑하는 사람이 애인이 있는데 난 그사람을 너무 사랑해요.,어떻게 하실 생각인가요?,그 사람과 그 애인이 헤어졌으면 좋겠어요.,그렇게하면 그 사람이 당신을 사랑하게 될까요?,그렇게 되게 만들거에요. 난 그사람을 너무 사랑해요.,당신이 너무 사랑하는게 느껴지네요.
1,9997,1,청년,여성,"연애, 결혼, 출산",해당없음,분노,분노,예전에 사귀었던 여자 친구가 결혼을 한다는 소식이 들려왔어.,그래요? 어차피 예전 여자 친구라서 상관없지 않나요?,응. 맞는데 너무 갑작스럽게 소식을 들었어. 차라리 몰랐으면 마음 편했을 건데.,그러게요. 마음이 좋지만은 않을 것 같아요.,꽤 멋진 놈이랑 결혼하는 것 같더군 사실 그 놈이 내 친구야. 그 둘 다 이제 내 ...,어째 그런일이. 충분히 화날만 할 것 같아요.
2,9996,1,청년,남성,"진로, 취업, 직장",해당없음,기쁨,자신하는,새로운 상사와 잘 지낼 수 있을 것 같아.,상사와의 관계에 자신이 있으시군요.,나와 취미가 잘 맞거든.,어떤 취미가 같나요?,축구를 좋아해.,잘 지내시길 바랄게요.
3,9995,1,청년,남성,"진로, 취업, 직장",해당없음,분노,악의적인,옆 팀 직원에게 악감정이 생겨.,악의적인 감정이 드시는군요.,퇴근하고 나서도 계속 생각이 나.,어떤 문제가 있나요?,나를 모함하고 다닌다는 말을 들었어.,힘내시길 바랄게요.
4,9994,1,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,나 면접 본 곳에서 내일부터 출근 하래! 너무 좋아!,좋은 소식이 찾아와서 기분이 좋으시겠어요.,응. 연락 올 줄 몰랐는데! 나도 이제 직장인이야!,회사에서는 어떤 일이 기다리고 있을까요?,힘든 일도 많겠지만 지금 기분 같아서는 다 잘할 수 있을 것만 같아! .,늘 응원하고 있을게요.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74851,7,1,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회의시간에 나의 직급이 낮아서 그런지 의견을 무시하는 경향이 있어서 의견을 제시하는...,의견을 무시당해서 의견을 내기가 힘드시군요.,무엇 때문에 그러는지 모르겠지만 변화를 해보고 싶어.,의견을 제시하기 위한 방법으로는 무엇이 있나요?,집에서 미리 생각해서 회의시간에 자신감 있게 말해야겠다.,회의시간에 의견을 내셔서 원하시는 결과를 얻으시길 바라요.
74852,6,1,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,내가 평소에 말이 좀 험한데 무서운 팀장님 앞에서 말실수를 할까 봐 걱정이야.,팀장님 앞에서 말실수를 할까 봐 걱정이시군요.,팀장님 앞에서만 있으면 떨리는데 괜찮을지 몰라.,말실수 안 하는 법은 무엇이 있나요?,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.
74853,3,1,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회사에 직장상사가 무섭게 생겨서 친해지는 게 두려워.,무섭게 생기셔서 친해지시는 게 어렵군요.,무섭게 생겼는데도 업무를 보려면 친해져야 할 필요가 있을 것 같아.,친해질 수 있는 방법은 무엇이 있을까요?,먼저 다가가서 말을 걸어볼게.,직장상사와 친해지시면 좋겠네요.
74854,2,1,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회사에서 중요한 큰 프로젝트를 나 혼자 하게 되니 솔직히 두려워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,내게 너무 크게 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,원하시는 결과가 나오면 좋겠네요.


In [ ]:
model3_data1 = model3_data[['사람문장1', '시스템응답1', '감정_대분류']]
model3_data1

,사람문장1,시스템응답1,감정_대분류
0,제가 사랑하는 사람이 애인이 있는데 난 그사람을 너무 사랑해요.,어떻게 하실 생각인가요?,분노
1,예전에 사귀었던 여자 친구가 결혼을 한다는 소식이 들려왔어.,그래요? 어차피 예전 여자 친구라서 상관없지 않나요?,분노
2,새로운 상사와 잘 지낼 수 있을 것 같아.,상사와의 관계에 자신이 있으시군요.,기쁨
3,옆 팀 직원에게 악감정이 생겨.,악의적인 감정이 드시는군요.,분노
4,나 면접 본 곳에서 내일부터 출근 하래! 너무 좋아!,좋은 소식이 찾아와서 기분이 좋으시겠어요.,기쁨
...,...,...,...
74851,회의시간에 나의 직급이 낮아서 그런지 의견을 무시하는 경향이 있어서 의견을 제시하는...,의견을 무시당해서 의견을 내기가 힘드시군요.,불안
74852,내가 평소에 말이 좀 험한데 무서운 팀장님 앞에서 말실수를 할까 봐 걱정이야.,팀장님 앞에서 말실수를 할까 봐 걱정이시군요.,불안
74853,회사에 직장상사가 무섭게 생겨서 친해지는 게 두려워.,무섭게 생기셔서 친해지시는 게 어렵군요.,불안
74854,회사에서 중요한 큰 프로젝트를 나 혼자 하게 되니 솔직히 두려워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,불안


In [ ]:
model3_data1.loc[(model3_data1['감정_대분류']=='불안'), '감정_대분류'] = 0
model3_data1.loc[(model3_data1['감정_대분류']=='당황'), '감정_대분류'] = 1
model3_data1.loc[(model3_data1['감정_대분류']=='분노'), '감정_대분류'] = 2
model3_data1.loc[(model3_data1['감정_대분류']=='슬픔'), '감정_대분류'] = 3
# model3_data.loc[(model3_data['감정_대분류']=='중립'), '감정_대분류'] = 4
model3_data1.loc[(model3_data1['감정_대분류']=='기쁨'), '감정_대분류'] = 5
model3_data1.loc[(model3_data1['감정_대분류']=='상처'), '감정_대분류'] = 6
model3_data1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,사람문장1,시스템응답1,감정_대분류
0,제가 사랑하는 사람이 애인이 있는데 난 그사람을 너무 사랑해요.,어떻게 하실 생각인가요?,2
1,예전에 사귀었던 여자 친구가 결혼을 한다는 소식이 들려왔어.,그래요? 어차피 예전 여자 친구라서 상관없지 않나요?,2
2,새로운 상사와 잘 지낼 수 있을 것 같아.,상사와의 관계에 자신이 있으시군요.,5
3,옆 팀 직원에게 악감정이 생겨.,악의적인 감정이 드시는군요.,2
4,나 면접 본 곳에서 내일부터 출근 하래! 너무 좋아!,좋은 소식이 찾아와서 기분이 좋으시겠어요.,5
...,...,...,...
74851,회의시간에 나의 직급이 낮아서 그런지 의견을 무시하는 경향이 있어서 의견을 제시하는...,의견을 무시당해서 의견을 내기가 힘드시군요.,0
74852,내가 평소에 말이 좀 험한데 무서운 팀장님 앞에서 말실수를 할까 봐 걱정이야.,팀장님 앞에서 말실수를 할까 봐 걱정이시군요.,0
74853,회사에 직장상사가 무섭게 생겨서 친해지는 게 두려워.,무섭게 생기셔서 친해지시는 게 어렵군요.,0
74854,회사에서 중요한 큰 프로젝트를 나 혼자 하게 되니 솔직히 두려워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,0


In [ ]:
# 오류(SettingWithCopyError 발생)
pd.set_option('mode.chained_assignment', 'raise') # SettingWithCopyError

# 경고(SettingWithCopyWarning 발생, 기본 값입니다)
pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning

# 무시
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

In [ ]:
model3_data2 = model3_data[['사람문장2', '시스템응답2', '감정_대분류']]
model3_data2.rename(columns={'사람문장2':'사람문장1', '시스템응답2':'시스템응답1'}, inplace=True)
# model3_data2
model3_data2.loc[(model3_data2['감정_대분류']=='불안'), '감정_대분류'] = 0
model3_data2.loc[(model3_data2['감정_대분류']=='당황'), '감정_대분류'] = 1
model3_data2.loc[(model3_data2['감정_대분류']=='분노'), '감정_대분류'] = 2
model3_data2.loc[(model3_data2['감정_대분류']=='슬픔'), '감정_대분류'] = 3
# model3_data.loc[(model3_data['감정_대분류']=='중립'), '감정_대분류'] = 4
model3_data2.loc[(model3_data2['감정_대분류']=='기쁨'), '감정_대분류'] = 5
model3_data2.loc[(model3_data2['감정_대분류']=='상처'), '감정_대분류'] = 6
model3_data2

,사람문장1,시스템응답1,감정_대분류
0,그 사람과 그 애인이 헤어졌으면 좋겠어요.,그렇게하면 그 사람이 당신을 사랑하게 될까요?,2
1,응. 맞는데 너무 갑작스럽게 소식을 들었어. 차라리 몰랐으면 마음 편했을 건데.,그러게요. 마음이 좋지만은 않을 것 같아요.,2
2,나와 취미가 잘 맞거든.,어떤 취미가 같나요?,5
3,퇴근하고 나서도 계속 생각이 나.,어떤 문제가 있나요?,2
4,응. 연락 올 줄 몰랐는데! 나도 이제 직장인이야!,회사에서는 어떤 일이 기다리고 있을까요?,5
...,...,...,...
74851,무엇 때문에 그러는지 모르겠지만 변화를 해보고 싶어.,의견을 제시하기 위한 방법으로는 무엇이 있나요?,0
74852,팀장님 앞에서만 있으면 떨리는데 괜찮을지 몰라.,말실수 안 하는 법은 무엇이 있나요?,0
74853,무섭게 생겼는데도 업무를 보려면 친해져야 할 필요가 있을 것 같아.,친해질 수 있는 방법은 무엇이 있을까요?,0
74854,내게 너무 크게 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,0


In [ ]:
model3_data3 = model3_data[['사람문장3', '시스템응답3', '감정_대분류']]
model3_data3.rename(columns={'사람문장3':'사람문장1', '시스템응답3':'시스템응답1'}, inplace=True)
# model3_data2
model3_data3.loc[(model3_data3['감정_대분류']=='불안'), '감정_대분류'] = 0
model3_data3.loc[(model3_data3['감정_대분류']=='당황'), '감정_대분류'] = 1
model3_data3.loc[(model3_data3['감정_대분류']=='분노'), '감정_대분류'] = 2
model3_data3.loc[(model3_data3['감정_대분류']=='슬픔'), '감정_대분류'] = 3
# model3_data.loc[(model3_data['감정_대분류']=='중립'), '감정_대분류'] = 4
model3_data3.loc[(model3_data3['감정_대분류']=='기쁨'), '감정_대분류'] = 5
model3_data3.loc[(model3_data3['감정_대분류']=='상처'), '감정_대분류'] = 6
model3_data3

,사람문장1,시스템응답1,감정_대분류
0,그렇게 되게 만들거에요. 난 그사람을 너무 사랑해요.,당신이 너무 사랑하는게 느껴지네요.,2
1,꽤 멋진 놈이랑 결혼하는 것 같더군 사실 그 놈이 내 친구야. 그 둘 다 이제 내 ...,어째 그런일이. 충분히 화날만 할 것 같아요.,2
2,축구를 좋아해.,잘 지내시길 바랄게요.,5
3,나를 모함하고 다닌다는 말을 들었어.,힘내시길 바랄게요.,2
4,힘든 일도 많겠지만 지금 기분 같아서는 다 잘할 수 있을 것만 같아! .,늘 응원하고 있을게요.,5
...,...,...,...
74851,집에서 미리 생각해서 회의시간에 자신감 있게 말해야겠다.,회의시간에 의견을 내셔서 원하시는 결과를 얻으시길 바라요.,0
74852,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.,0
74853,먼저 다가가서 말을 걸어볼게.,직장상사와 친해지시면 좋겠네요.,0
74854,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,원하시는 결과가 나오면 좋겠네요.,0


In [ ]:
model3_data = pd.concat([model3_data1, model3_data2, model3_data3])
model3_data

,사람문장1,시스템응답1,감정_대분류
0,제가 사랑하는 사람이 애인이 있는데 난 그사람을 너무 사랑해요.,어떻게 하실 생각인가요?,2
1,예전에 사귀었던 여자 친구가 결혼을 한다는 소식이 들려왔어.,그래요? 어차피 예전 여자 친구라서 상관없지 않나요?,2
2,새로운 상사와 잘 지낼 수 있을 것 같아.,상사와의 관계에 자신이 있으시군요.,5
3,옆 팀 직원에게 악감정이 생겨.,악의적인 감정이 드시는군요.,2
4,나 면접 본 곳에서 내일부터 출근 하래! 너무 좋아!,좋은 소식이 찾아와서 기분이 좋으시겠어요.,5
...,...,...,...
74851,집에서 미리 생각해서 회의시간에 자신감 있게 말해야겠다.,회의시간에 의견을 내셔서 원하시는 결과를 얻으시길 바라요.,0
74852,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.,0
74853,먼저 다가가서 말을 걸어볼게.,직장상사와 친해지시면 좋겠네요.,0
74854,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,원하시는 결과가 나오면 좋겠네요.,0


In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/asset/test_data/감성대화말뭉치AI데이터_평가용Wave_대본1만문장.csv')
test2_data = pd.read_csv('/content/drive/MyDrive/asset/test_data/emotion_data.csv')
test2_data

In [ ]:
test_data = test_data[['사람문장1']]
test_data
test_data.iloc[0]

In [ ]:
test1_data = []
for i in range(len(test_data)) :
  if i % 50 == 0 :
    test1_data.append(test_data.iloc[i][0])
test1_data

['오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.',
 '할머니 집 마당에서 다 같이 고기를 구워먹었는데 참 행복했어.',
 '문제를 다 풀면 개념은 확실해질 거라 믿어.',
 '내 느낌에 이번 모의고사 평균 이 등급 받을 수 있을 거 같아!',
 '방금 내가 틀린 줄 알았던 문제의 답이 복수 정답 처리가 되었다는 말을 들었어. 너무 기뻐.',
 '고장 난 노트북을 고치는 대신 아빠가 최신형으로 바꿔 주셔서 기뻐.',
 '수능 성적이 안 좋아서 심란했는데 아빠가 위로해주셔서 기뻤어.',
 '친구들이 나를 괴롭히지 못하도록 선생님께서 감시하신다고 하셔서 얼마나 다행인지 몰라.',
 '오늘 기말고사를 봤는데 공부했던 부분에서 문제가 다 나와서 기분이 좋아.',
 '오늘 학교에 아빠가 작업복을 입고 면담을 오셨는데 옷차림이 신경이 쓰이더라.',
 '형이 없으면 나도 가족들에게 인정받을 수 있을 거야.',
 '별 것도 아닌 일로 내가 친구를 때렸어. 이제 생각하면 너무 부끄러워.',
 '우리 반에서 어느 한 명을 따돌림 시키는 일이 정말 옳은 일인가 싶어.',
 '내가 날씬해지면 인기가 많아질 것 같아.',
 '시험을 쳤는데 성적이 나빠 부모님께 성적표를 보여드리기 부끄러워.',
 '내가 너무 솔직하게 말을 한 걸까? 요즘 학교에 가기 싫어.',
 '정답지에 공부했던 내용을 전부 완벽히 쓰지 못해 나한테 화가 나.',
 '내 인사도 무시하던 친구가 내가 전교 일등 했다는 소식을 듣고 갑자기 친한 척을 해서 불쾌해.',
 '선생님은 친한 친구랑 나를 경쟁하게 만들어서 마음이 아파',
 '나한테 욕을 퍼붓는 애가 있는데 왜 성격이 나쁜데도 걔 옆에는 친구들이 많은지 궁금해.',
 '오늘 순간 욱해서 친구를 때렸어. 나 같은 구제 불능은 어디서도 안 받아줄 거야.',
 '오빠가 자기 인생 살겠다고 집을 나갔으면서 돈이 필요하니까 다시 들어왔어. 정말 구역질 나.',
 '친한 친구와 경쟁심이 생겨 공부 얘기를 잘하지 않게 되었어.',
 '노는 애

In [ ]:
col_n = "사람문장1"
test1_datap = pd.DataFrame(test1_data, columns = [col_n])
test1_datap

,사람문장1
0,오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.
1,할머니 집 마당에서 다 같이 고기를 구워먹었는데 참 행복했어.
2,문제를 다 풀면 개념은 확실해질 거라 믿어.
3,내 느낌에 이번 모의고사 평균 이 등급 받을 수 있을 거 같아!
4,방금 내가 틀린 줄 알았던 문제의 답이 복수 정답 처리가 되었다는 말을 들었어. 너...
...,...
195,지금은 건강하지만 나중에 아프게 되면 걱정인데 죽음이후에 사후세계가 있을까.
196,자녀들이 암에 걸린 나를 서로 돌보지 않겠다고 싸우고 있어. 환멸이 나.
197,담배를 좀 더 일찍 끊었으면 좋았을 텐데 후회가 되네.
198,은퇴 후에 해보고 싶었던 일로 첫 도전을 했던 마라톤에 참여하여 완주해서 정말 기뻐.


In [ ]:
test_data.iloc[0][0]

'오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.'

In [ ]:
model3_data = model3_data.dropna()
print(model3_data.isnull().sum())

사람문장1     0
시스템응답1    0
감정_대분류    0
dtype: int64


In [ ]:
model3_data

,사람문장1,시스템응답1,감정_대분류
0,제가 사랑하는 사람이 애인이 있는데 난 그사람을 너무 사랑해요.,어떻게 하실 생각인가요?,2
1,예전에 사귀었던 여자 친구가 결혼을 한다는 소식이 들려왔어.,그래요? 어차피 예전 여자 친구라서 상관없지 않나요?,2
2,새로운 상사와 잘 지낼 수 있을 것 같아.,상사와의 관계에 자신이 있으시군요.,5
3,옆 팀 직원에게 악감정이 생겨.,악의적인 감정이 드시는군요.,2
4,나 면접 본 곳에서 내일부터 출근 하래! 너무 좋아!,좋은 소식이 찾아와서 기분이 좋으시겠어요.,5
...,...,...,...
74851,집에서 미리 생각해서 회의시간에 자신감 있게 말해야겠다.,회의시간에 의견을 내셔서 원하시는 결과를 얻으시길 바라요.,0
74852,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.,0
74853,먼저 다가가서 말을 걸어볼게.,직장상사와 친해지시면 좋겠네요.,0
74854,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,원하시는 결과가 나오면 좋겠네요.,0


In [ ]:
mask = model3_data['감정_대분류'].isin(['ㅈ중립', '분ㄴ', '중림', 'ㅍ', 'ㄴ중립', '분', '줄'])
model3_data = model3_data[~mask]
model3_data

,사람문장1,시스템응답1,감정_대분류
0,제가 사랑하는 사람이 애인이 있는데 난 그사람을 너무 사랑해요.,어떻게 하실 생각인가요?,2
1,예전에 사귀었던 여자 친구가 결혼을 한다는 소식이 들려왔어.,그래요? 어차피 예전 여자 친구라서 상관없지 않나요?,2
2,새로운 상사와 잘 지낼 수 있을 것 같아.,상사와의 관계에 자신이 있으시군요.,5
3,옆 팀 직원에게 악감정이 생겨.,악의적인 감정이 드시는군요.,2
4,나 면접 본 곳에서 내일부터 출근 하래! 너무 좋아!,좋은 소식이 찾아와서 기분이 좋으시겠어요.,5
...,...,...,...
74851,집에서 미리 생각해서 회의시간에 자신감 있게 말해야겠다.,회의시간에 의견을 내셔서 원하시는 결과를 얻으시길 바라요.,0
74852,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.,0
74853,먼저 다가가서 말을 걸어볼게.,직장상사와 친해지시면 좋겠네요.,0
74854,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,원하시는 결과가 나오면 좋겠네요.,0


In [ ]:
model3_data['감정_대분류'].unique()
# test1_datap['사람문장1'] = test1_datap['사람문장1'].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

# concat_data['감정_대분류'].value_counts()

array([2, 5, 1, 6, 0, 3], dtype=object)

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["사람문장1"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["시스템응답1"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.
        sentiment = str(turn['감정_대분류'])
        q_toked = self.tokenizer.tokenize(self.q_token + q + \
                                          self.sent_token + sentiment)  
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [ ]:
train_set = ChatbotDataset(model3_data, max_len=60)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [ ]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
print("end")

start


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  """


end


In [ ]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
learning_rate = 5e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none").cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 6
Sneg = -1e18


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.long().to(device)
        label = label.long().to(device)
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2).to(device)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out)).to(device)
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

start


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/6230 [00:00<?, ?it/s]

  0%|          | 0/6230 [00:00<?, ?it/s]

  0%|          | 0/6230 [00:00<?, ?it/s]

  0%|          | 0/6230 [00:00<?, ?it/s]

  0%|          | 0/6230 [00:00<?, ?it/s]

  0%|          | 0/6230 [00:00<?, ?it/s]

end


In [ ]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
torch.save(model.state_dict() , '/content/drive/MyDrive/Model3(Text Generation)/model3_61.pt')

transformer.wte.weight 	 torch.Size([51200, 768])
transformer.wpe.weight 	 torch.Size([1024, 768])
transformer.h.0.ln_1.weight 	 torch.Size([768])
transformer.h.0.ln_1.bias 	 torch.Size([768])
transformer.h.0.attn.bias 	 torch.Size([1, 1, 1024, 1024])
transformer.h.0.attn.masked_bias 	 torch.Size([])
transformer.h.0.attn.c_attn.weight 	 torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias 	 torch.Size([2304])
transformer.h.0.attn.c_proj.weight 	 torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias 	 torch.Size([768])
transformer.h.0.ln_2.weight 	 torch.Size([768])
transformer.h.0.ln_2.bias 	 torch.Size([768])
transformer.h.0.mlp.c_fc.weight 	 torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias 	 torch.Size([3072])
transformer.h.0.mlp.c_proj.weight 	 torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias 	 torch.Size([768])
transformer.h.1.ln_1.weight 	 torch.Size([768])
transformer.h.1.ln_1.bias 	 torch.Size([768])
transformer.h.1.attn.bias 	 torch.Size([1, 1, 1024, 1024])
tr

In [ ]:
a_list = []
with torch.no_grad():
    for i in range(200):
        q = input()
        sent = str(5)
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).cpu().squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))
        a_list.append(a.strip())
a_list

Chatbot > 강아지가 정말 좋아하시는군요
Chatbot > 강아지를 좋아하시는군요  강아지를 좋아하시는 이유가 있나요
Chatbot > 맞아요  귀엽지 않아요
Chatbot > 좋은 사랑 하셨으면 좋겠네요


KeyboardInterrupt: ignored